# Final Project CS203 - LOAD VALUE PREDICTION

---

- Students: Shreya Singh, Vincent Tran. 

# Model
Long Short-Term Memory, a type of Recurrent Neural Network. This Neural Network allows previous outputs to be used as inputs while having hidden states. 

# Why we use this
RNNs can take one or more input vectors and produce one or more output vectors and the output(s) are influenced not just by weights applied on inputs like a regular NN, but also by a “hidden” state vector representing the context based on prior input(s)/output(s). In order to remedy the vanishing gradient problem of RNNs, LSTM model is used. LSTM (Long Short Term Memory) networks improve on the RNN model by introducing additional gates and a cell state, such that it fundamentally addresses the problem of keeping or resetting context. It is a solution to the vanishing gradient problem. RNN finds its application when we are dealing with sequential data or data with temporal relationships.

# How we create the model

- In our model, we split the data set by training set and test set. The PC address of load instruction memory, memory address, and value in the training set are used to build the model. (We only fed the Load values in the model) Then we feed the load instruction memory and memory address  into the model to get the predicted value. We compare the predicted value and actual value based on the confidence value(use Mean Squared Errors as the threshold) to decide whether or not we should choose the data. 

- If the square of the difference between actual and predicted value is less than the Mean Square Errors, we choose the predicted value, otherwise we don’t.

- If the predicted value is a bit off, we update the model by changing the model parameters such as epochs and the size of training set, training window.
If the prediction is close to actual, we keep our model. 

- Our prediction model based on the recent instances. 

# Model:
- Input: 
  - The actual values at the memory address for training.
  - Because the actual values are stored as long range hexadecimal, we convert the hexadecimal to integer.
- Ouput: The predicted Value on the test data.

# Limitation:
- Because the training data is big, it would take a long time to build the model. 
- In our model, we are able to get the results with small data set (10000), and smaller number of epochs(60).

# Feature work
- Increasing number of epochs and using a larger dataset to improve the model. 



# This is the coding for CS203 final project

---



In [ ]:
#!wget https://drive.google.com/drive/folders/1b1cqriOkf87Qb55WPyw04jrdM4Hths_X?usp=sharing
#!wget https://drive.google.com/file/d/1xY1M8zelUpYNXI-wGgTEpU9ABKaYiBZX/view?usp=sharing
#!wget https://drive.google.com/file/d/1RjFeKS2xJbGoOc1Id-DI-MhKnZw20flO/view?usp=sharing

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

# Upload the Data into the Colab folder

---

- The data must be raw data, and the format file must be .csv
- the data should have three colums: Load Instruction PC, memory address, and value.
- For example:  
0x7f8545bfde1f:  R  0x7f8545c29e68,   0x000000000000000e

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
     name=fn, length=len(uploaded[fn])))

TypeError: ignored

# Clean up the data

---

- Replace `:` by `,`
- Remove 'R'
- Note: In this project, we use the file name: pinatrace1.csv. If your file is different, please feel free to update. 


In [ ]:
import fileinput
import re
filename = "pinatrace1.csv" # update your file name here

with open(filename, 'r+') as f:
    text = f.read()
    text = re.sub(':',',',text)
    text = re.sub('R','', text)
    #text = text.replace('0x','')
    f.seek(0)
    f.write(text)
    f.truncate()

# Add the Library

---



In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler
from ast import literal_eval 

# Process the Data

---

- We convert the value from hexadecimal to decimal number for training model puporses. 
- We remove all the rows that contain #eof, NaN
- We remove all duplicate rows.
- Columns name:
  - PCAddr: PC address of load instruction.
  - memAddr: memory address of load instruction.
  - value: the value of load instruction.

In [ ]:
data = pd.read_csv('pinatrace1.csv',names=['PCAddr','memAddr','value'])
#data.head(5)
data.sort_values(by=['PCAddr','memAddr'])
print(data.size)
#print(data.head(5))
duplicate = data[data.duplicated(['PCAddr','memAddr','value'])]
#print(duplicate,sep='\n')
print("duplicate: ", duplicate.size)
#data.duplicated()
data = data.drop_duplicates()
print("after remove duplicate " , data.size)
#print(data.info())
#type(data)
data.drop(data.tail(1).index)
data.dropna(subset=['memAddr'],inplace=True)
#print(data.loc[[531885]])

# Drop the last n rows in data
#376123 - 351123 = 25000
print(data.info())


# Reduce the data

In [ ]:
# Input number of entries 
entries = input("Please enter number entries for data: ")

In [ ]:
total = data.size/3;
remove_n = int(total - (int)(entries))
#df = pd.DataFrame({"a":[1,2,3,4], "b":[5,6,7,8]})
drop_indices = np.random.choice(data.index, remove_n, replace=False)
data = data.drop(drop_indices)

totalRows = total - remove_n
#n = 351123
#data.drop(data.tail(n).index, 
#        inplace = True) 
print(data.info())

In [ ]:
#print(type(data['memAddr']))
data['value'] = data['value'].str.strip()
#print(type(s))
s = data['value']
counter = 0
ls = list()
#s.drop(s.tail(1))
for i in s:
  #print(type(i))
  #test_string = i
  #print(i)
  #if(i is 'nan'):
  #  break
  i = literal_eval(str(i))
  ls.append(i)
  #print("Counter: ", counter)
  #counter = counter + 1
  #print("i: ", i)
  #print("The decimal number of hexadecimal string : " + str(literal_eval(str(i))))
  #data['memAddr'].replace(i,res)
  
#print(data['memAddr'])

# Convert address to integer, back and forth
# Python3 code to demonstrate 
# converting hexadecimal string to decimal 
# Using ast.literal_eval() 

# initializing string  
#test_string = '0x7f8545c29e68'
#print(type(test_string))
# printing original string  
#print("The original string : " + str(test_string)) 
  
# using ast.literal_eval() 
# converting hexadecimal string to decimal 
#res = literal_eval(test_string) 

# print result 
#print("The decimal number of hexadecimal string : " + str(res)) 
#print(hex(res))



In [ ]:
print(type(data))
print(data.head(5))
print(data.info())

In [ ]:
data.columns

In [ ]:
print(data.size)

In [ ]:
print(type(s))

In [ ]:
data['valueInDec'] = pd.Series(ls, index=data.index)

In [ ]:
print(data.head(5))

In [ ]:
# Convert the value in Decimal to hex
decToAddr = list()
for i in ls:
  decToAddr.append(hex((i)))

data['Value'] = pd.Series(decToAddr,index=data.index)
print(data.head(5))

In [ ]:
all_data = pd.DataFrame()
all_data['PCAddr'] = data['PCAddr']
all_data['memAddr']= data['memAddr']
all_data['value'] = data['valueInDec']
print(all_data.head(5))

# Process the Reinforcement Learning- LSTM starts here


---



## Check data type of each columns in the data

In [ ]:
all_data.columns
print(all_data.info())

## Convert the value to decimal number

In [ ]:
all_data = all_data['value'].values.astype(float) # convert object to float for value columns
print(type(all_data))

In [ ]:
print(all_data)
print(len(all_data))

## Split the data into the test data and training data

In [ ]:
# Split data by percent
splitRate = input("Please enter percent of data that will be test")

In [ ]:
test_data_size = (int)((float)(splitRate) * totalRows) # Total is 376123, after reducing data it is 25000 rows

train_data = all_data[:-test_data_size]
test_data = all_data[-test_data_size:]
print(type(train_data))
print(type(test_data))
print(len(train_data))
print(len(test_data))
print(test_data)

## Normalize the value columns

---

- the feature_range contains positive number only.
- We will perform min/max scaling on the dataset which normalizes the data within a certain range of minimum and maximum values
- [When the feature_range is symmetric (-1,1), we will get the negative number in ouput. Hence, the predicted value is negative which is impossible in the actual value. ]


In [ ]:
scaler = MinMaxScaler(feature_range=(0,1000)) # Need to check the range
train_data_normalized = scaler.fit_transform(train_data.reshape(-1,1))
print(train_data_normalized[:5])
print(train_data_normalized[-5:0])

## Convert value into tensors because PyTorch models are trained using tensors.

In [ ]:
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)
train_window = (int)((float)(splitRate) * totalRows)

## Create the input sequence function

- input: 
  - raw data
  - train window
- output: tuple
  - first element is array contains where its length is train window.
  - second element is first element in the next train_window.


In [ ]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw:i+tw+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [ ]:
train_inout_seq = create_inout_sequences(train_data_normalized, train_window)
train_inout_seq[:4]

# Creating LSTM Model

---

- input_size: Corresponds to the number of features in the input. In this case, we use only the values so the input_size = 1. If we use 2 features: PC, memory load, then the input_size = 2.
- hidden_layer_size: Specifies the number of hidden layers along with the number of neurons in each layer. The default is 100.
- output_size: the number of items in the output, output_size = 1 is the predited valued. It is the predicted value.


In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_layer_size=100, output_size=1):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size),
                            torch.zeros(1,1,self.hidden_layer_size))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]


In [ ]:
model = LSTM()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print(model)

# Training the Model

---

- The size of data is large, so if we increase the number of epochs, then it takes a long time to training the model with GPU.
- In our experiment, the number of epochs varies from 10 to 60.

In [ ]:
epochs =  input("Please enter number of epochs: ")

In [ ]:
#print(train_inout_seq)
#for i in range(epochs):
  
for seq, labels in train_inout_seq:
  optimizer.zero_grad()
  model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))

  y_pred = model(seq)
        #print("inside")
  single_loss = loss_function(y_pred, labels)
  single_loss.backward()
  optimizer.step()

  if i%25 == 1:
      print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

# Making Predictions

---

- We filtered 20% values from the data, and print out the output


In [ ]:
fut_pred = (int)(0.2 * totalRows)

test_inputs = train_data_normalized[-train_window:].tolist()
print(test_inputs)
print(len(test_inputs))

### Make Prediction

---

- The outer for loop will execute fut_pred times.
- In the second iterator, the last tuples will be used as the input and a new prediction will be made which will then be appended to the test_inputs list again. 

In [ ]:
model.eval()

for i in range(fut_pred):
    seq = torch.FloatTensor(test_inputs[-train_window:])
    with torch.no_grad():
        model.hidden = (torch.zeros(1, 1, model.hidden_layer_size),
                        torch.zeros(1, 1, model.hidden_layer_size))
        test_inputs.append(model(seq).item())

Because the training data is normalized, then the result is also normalized. We must transform the results back to the normal value using inverse_transform, and print the actual_prediction.

In [ ]:
test_inputs[fut_pred:]
actual_predictions = scaler.inverse_transform(np.array(test_inputs[train_window:] ).reshape(-1, 1))
print(actual_predictions)
print(len(actual_predictions))

# Evaluate the results

---

- Actual value is last 20 percent of the total rows in all_data
- Calculate the mean square errors
- calculate square of difference between the predicted value and actual value
- compare the Mean Square Error with the squared difference. 
  - If MSE < difference: do not choose the predicted value, Mark as 0
  - if MSE >= difference: choose the predicted value, Mark as 1


In [ ]:
print(len(test_data))
print(len(actual_predictions))

In [ ]:
# Actual prediction is the predicted value after transform
# y_test = test_data
#train_data = all_data[:-test_data_size]
#test_data = all_data[-test_data_size:]

#y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
#y_train = scaler.inverse_transform(y_train.detach().numpy())
#y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
#y_test = scaler.inverse_transform(y_test.detach().numpy())

# calculate root mean squared error
#trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
#print('Train Score: %.2f MSE' % (trainScore))
#testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
#print('Test Score: %.2f MSE' % (testScore))
import math
from sklearn.metrics import mean_squared_error
#trainScore = mean_squared_error(test_data, actual_predictions)
#trainScore = math.sqrt(mean_squared_error(test_data, actual_predictions))
#print('Train Score: %.2f MSE', (trainScore))
threshold = (mean_squared_error(test_data, actual_predictions))
print('Test Score: %.2f MSE' , (threshold))


# Print the PC, Memory Address, Actual Value(hex), Predicted Value, Decision


# Plot the results

In [ ]:
from matplotlib import pyplot as plt
# Display the x value, PC and Memory address
t = data.tail(len(test_data))
decision = list()
MSE = list()
for i in range(len(test_data)):
  if math.pow(test_data[i] - actual_predictions[i],2) > threshold:
    MSE.append(math.pow(test_data[i] - actual_predictions[i],2))
    decision.append('NO')
  else:
    MSE.append(math.pow(test_data[i] - actual_predictions[i],2))
    decision.append('YES')

#t['x'] = x
#print(test_data)
bases = np.amin(actual_predictions)
#print(bases)
t['Decision'] = decision
#t['test_data'] = np.true_divide(test_data,bases)
#t['actual_predictions'] = np.true_divide(actual_predictions ,bases)
print(t)
print(t.info())

# print number of yes (choose the value) and no (don't choose the value)

In [ ]:
print(t['Decision'].describe(include=['category']))
print("Size: ", len(all_data))
print("training size: ", len(train_data))
print("test size: ", len(test_data))
print("epochs: ", epochs)

# Plot the threshold and mean square error for each load PC, and memory address

---



## Plot the test value, threshold and prediction value of each load PC and memory address

In [ ]:
plt.title('PC Address, Memory Address - Predicted value and actual value')
plt.ylabel('Value')
plt.grid(True)
plt.autoscale(axis='PC Address, Memory Address', tight=True)
x_length = len(test_data)
x = list(range(0,x_length))
#plt.plot(x,test_data, color="red")
#plt.plot(x,actual_predictions,color="blue")
plt.plot(x,np.true_divide(test_data,bases), color="red")
plt.plot(x,np.true_divide(actual_predictions,bases),color="blue")
#plt.plot(x,np.array([threshold for i in range(len(test_data))]),color="Green")
plt.show()

## plot the threshold and Squared difference of actual value and predicted value of each load PC and memory address.

In [ ]:
plt.title('PC Address, Memory Address - Predicted value and actual value')
plt.ylabel('Value')
plt.grid(True)
plt.autoscale(axis='PC Address, Memory Address', tight=True)
x_length = len(test_data)
x = list(range(0,x_length))
plt.plot(x,MSE, color="red")
plt.plot(x,np.array([threshold for i in range(len(test_data))]),color="Green")
plt.show()